# Imports

In [1]:
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np
from sklearn.datasets import make_classification
import csv
import pickle
import math
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import validation_curve
from sklearn.datasets import load_iris
from sklearn.linear_model import Ridge
from sklearn import preprocessing
from dtw import dtw
from fastdtw import fastdtw
import sys
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import os
import json
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from flask import Flask, render_template, request, Response

# Functions

In [2]:
def getlabel (Jsonobj , sensor):
    LABEL=[]
    acc=Jsonobj.get(sensor)
    for i in range (len(acc)):
        LABEL.append(acc[i].get('patternDotNum'))
    return LABEL

In [3]:
def getlabelgyro (Jsonobj , sensor):
    LABELgyro=[]
    gyro=Jsonobj.get(sensor)
    for i in range (len(gyro)):
        LABELgyro.append(gyro[i].get('patternDotNum'))
    return LABELgyro

In [4]:
def readJson(JsonObj , sensor):
    MOALLX=[]
    MOALLY=[]
    MOALLZ=[]
    MOALLANG=[]
    acc=JsonObj.get(sensor)
    for i in range (len(acc)):
        MOALLX.append(acc[i].get('x'))
        MOALLY.append(acc[i].get('y'))
        MOALLZ.append(acc[i].get('z'))
        MOALLANG.append(acc[i].get('angle'))
        
    return reshaping2(MOALLX,MOALLY,MOALLZ,MOALLANG)

In [5]:
def readJson(JsonObj , sensor):
    MOALLX=[]
    MOALLY=[]
    MOALLZ=[]
    MOALLANG=[]
    acc=JsonObj.get(sensor)
    for i in range (len(acc)):
        MOALLX.append(acc[i].get('x'))
        MOALLY.append(acc[i].get('y'))
        MOALLZ.append(acc[i].get('z'))
        MOALLANG.append(acc[i].get('angle'))
        
    return reshaping2(MOALLX,MOALLY,MOALLZ,MOALLANG)

In [6]:
def reshaping5gyro (X , Y , Z ):
    myarray=[]
    print(len(X))
    for i in range (len(X)):
        temp=[X[i],Y[i],Z[i]]
        myarray.append(temp)
    return myarray

In [7]:
def readJson1(JsonObj, sensor ):
    MOALLX=[]
    MOALLY=[]
    MOALLZ=[]
    MOALLANG=[]
    MOALLPL=[]
    acc=JsonObj.get('acc')
    for i in range (len(acc)):
        MOALLX.append(acc[i].get('x'))
        MOALLY.append(acc[i].get('y'))
        MOALLZ.append(acc[i].get('z'))
        MOALLANG.append(acc[i].get('angle'))
        MOALLPL.append(acc[i].get('patternDotNum'))
        
    return reshaping5(MOALLX,MOALLY,MOALLZ,MOALLANG,MOALLPL)

In [8]:
def readJsongyro(JsonObj , sensor):
    MOALLX=[]
    MOALLY=[]
    MOALLZ=[]
    MOALLANG=[]
    MOALLPL=[]
    gyro=JsonObj.get(sensor)
    for i in range (len(gyro)):
        MOALLX.append(gyro[i].get('x'))
        MOALLY.append(gyro[i].get('y'))
        MOALLZ.append(gyro[i].get('z'))
        
    return reshaping5gyro(MOALLX,MOALLY,MOALLZ)

In [9]:
def reshaping2 (X , Y , Z , A):
    myarray=[]
    for i in range (len(X)):
        temp=[X[i],Y[i],Z[i],A[i]]
        myarray.append(temp)
    return myarray

In [10]:
def reshapinggyro (X , Y , Z ):
    myarray=[]
    for i in range (len(X)):
        temp=[X[i],Y[i],Z[i]]
        myarray.append(temp)
    return myarray

In [11]:
def lcs(X , Y):
    m = len(X)
    n = len(Y)
    L = [[None]*(n+1) for i in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j] , L[i][j-1])

    return L[m][n]


In [12]:
def access(percent):
    if percent < 60.00 :
        YN = "{'access':'NO'}"
    else:
        YN ="{'access':'YES'}"
        
    return YN

In [13]:
def readFileDataR(file):
    column_names = ['x','y','z','angle','patternDotNum']
    data = pd.read_csv(file)

    x = data['x']
    y = data['y']
    z = data['z']
    a = data['angle']
    pl= data['patternDotNum']

    return np.dstack([x,y,z,a,pl])[0]


In [14]:
def readFileDataRgyro(file):
    column_names = ['x','y','z','angle','patternDotNum']
    data = pd.read_csv(file)

    x = data['x']
    y = data['y']
    z = data['z']
    pl= data['patternDotNum']

    return np.dstack([x,y,z,pl])[0]

In [15]:
def combineToSave(feature,label):
    myarray=[]
    for i in range(len(feature)):
        myarray[i]=feature[i]+label[i]
    return myarray

In [16]:
def writeincsv(FileId,myarray):
    with open(FileId,'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['x','y','z','angle','patternDotNum'])
        csvwriter.writerows(myarray)
    return

In [17]:
def writeincsvgyro(FileId,myarray):
    with open(FileId,'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['x','y','z','patternDotNum'])
        csvwriter.writerows(myarray)
    return

# Traning

In [ ]:
app = Flask(__name__)

APP_ROOT = os.path.dirname(os.path.abspath('__file__'))

@app.route("/")
#def index():
#    return render_template("upload.html")

@app.route("/register", methods=['POST'])
def reg():
    target = os.path.join(APP_ROOT, 'dataset/')
    print(target)
    
    if not os.path.isdir(target):
        os.mkdir(target)
    
    jsonOReg = request.get_json()
    elmentNUM=jsonOReg.get('registration_array')
    #for acc==============================================================================================================
    sensoracc='acc'
    
    #extract feature 
    elm1=readJson(elmentNUM[0],sensoracc)
    elm2=readJson(elmentNUM[1],sensoracc)
    elm3=readJson(elmentNUM[2],sensoracc)
    
    feature=elm1+elm2+elm3
    
    #extract label
    label=[]
    LB1=getlabel(elmentNUM[0],sensoracc)
    LB2=getlabel(elmentNUM[1],sensoracc)
    LB3=getlabel(elmentNUM[2],sensoracc)
    
    label=LB1+LB2+LB3
        
    # In[8]:
    #all the data to save =================================================================================================
    data1=readJson1(elmentNUM[0],sensoracc)
    data2=readJson1(elmentNUM[1],sensoracc)
    data3=readJson1(elmentNUM[2],sensoracc)
    
    data=data1+data2+data3
    writeincsv('myreads.csv',data)
    model = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
    model.fit(feature, label)
    #modelGYRO=============================================================================================================
    sensorgyro='gyro'
    
    #extract featuregyro 
    elmgyro1=readJsongyro(elmentNUM[0],sensorgyro)
    elmgyro2=readJsongyro(elmentNUM[1],sensorgyro)
    elmgyro3=readJsongyro(elmentNUM[2],sensorgyro)
    
    featuregyro=elmgyro1+elmgyro2+elmgyro3
    
    #extract labelgyro
    labelgyro=[]
    LBgyro1=getlabelgyro(elmentNUM[0],sensorgyro)
    LBgyro2=getlabelgyro(elmentNUM[1],sensorgyro)
    LBgyro3=getlabelgyro(elmentNUM[2],sensorgyro)
    
    labelgyro=LBgyro1+LBgyro2+LBgyro3
        
    # In[8]:
    #all the data to save gyro =================================================================================================
    datagyro1=readJsongyro(elmentNUM[0],sensorgyro)
    datagyro2=readJsongyro(elmentNUM[1],sensorgyro)
    datagyro3=readJsongyro(elmentNUM[2],sensorgyro)
    
    datagyro=datagyro1+datagyro2+datagyro3
    print(data)
    writeincsvgyro('myreadsgyro.csv',datagyro)
    modelgyro = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
    print(featuregyro)
    print(labelgyro)
    modelgyro.fit(featuregyro, labelgyro)
    
    #read cSv file ========================================================================================================
    #
    pickle.dump(model, open('modle.p', 'wb'))
    pickle.dump(modelgyro, open('modlegyro.p', 'wb'))
    resp="{'permition':'ready for authantication'}"
    return Response(resp, status=201, mimetype='application/json')


# predicting

In [1]:
@app.route("/")
#def index():
#    return render_template("upload.html")

@app.route("/upload", methods=['POST'])
def upload():
    target = os.path.join(APP_ROOT, 'dataset/')
    print(target)

    X=[]
    Y=[]
    Z=[]
    A=[]
    
    Xg=[]
    Yg=[]
    Zg=[]

    if not os.path.isdir(target):
        os.mkdir(target)

    
    jsonO = request.get_json()
    #acc===================================================================================================================
    acc=jsonO.get('acc')
    #print(len(acc))
    for i in range (len(acc)):
        X.append(acc[i].get('x'))
        Y.append(acc[i].get('y'))
        Z.append(acc[i].get('z'))
        A.append(acc[i].get('angle'))
    #gyro==================================================================================================================
    gyro=jsonO.get('gyro')
    for i in range (len(gyro)):
        Xg.append(gyro[i].get('x'))
        Yg.append(gyro[i].get('y'))
        Zg.append(gyro[i].get('z'))
# # predect

# In[ ]:

    mypred=reshaping2(X,Y,Z,A)
    mypredgyro=reshapinggyro(Xg,Yg,Zg)
    # In[ ]:

    loaded_model = pickle.load(open('modle.p', 'rb'))
    loaded_modelgyro = pickle.load(open('modlegyro.p', 'rb'))
    # Visualisation
    # We will use 70 plots between minimum and maximum values of valid_x for plotting
    xp = mypred
    pred_plot =loaded_model.predict(xp)
    
    xpg=mypredgyro
    pred_plotg=loaded_modelgyro.predict(xpg)
    # In[ ]:
    del xp[:]
    del xpg[:]
    #print(len(roundedpre))
    #print(xp)
    #print(roundedpre)
    del X[:]
    del Y[:]
    del Z[:]
    del A[:]
    del Xg[:]
    del Yg[:]
    del Zg[:]
    MOALLR=readFileDataR("myreads.csv")
    MOALLPL=MOALLR[:, 4]
    MOALLR=readFileDataRgyro("myreadsgyro.csv")
    MOALLPLgyro=MOALLR[:, 3]
    resp = lcs(pred_plot,MOALLPL)
    respg = lcs(pred_plotg,MOALLPLgyro)
    percentacc = ((resp/len(acc)) * 100)
    percentgyro=((resp/len(gyro)) * 100)
    #print(percent)
    avgpercent=(percentacc+percentgyro)/2
    YN=access(avgpercent)
    print(avgpercent)
    return Response(YN, status=201, mimetype='application/json')


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=80)